IMDB 영화평 감성 분석(이진분류)

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('labeledTrainData.tsv',sep='\t')
df.head(2)

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."


In [3]:
df = pd.read_csv('labeledTrainData.tsv',sep='\t',quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


* 텍스트 전처리

In [4]:
#<br /> 태그 공백변환
df.review = df.review.str.replace('<br />',' ')

In [5]:
#구둣점, 숫자 제거 - 영문자가 아닌것은 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]',' ').str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [6]:
df.review[0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

* Train/Test dataset 분리

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
    df.review,df.sentiment,stratify=df.sentiment,random_state=2022
)

In [8]:
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

* CountVectorizer로 변환

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [11]:
cvect.fit(X_train)
X_train_cv = cvect.fit_transform(X_train)
X_train_cv.shape

(18750, 65213)

In [13]:
X_test_cv = cvect.transform(X_test)
X_test_cv.shape

(6250, 65213)

* 분류기 Naive Bayes

In [14]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [16]:
%time nb.fit(X_train_cv,y_train)

CPU times: user 31.1 ms, sys: 1 ms, total: 32.1 ms
Wall time: 32.4 ms


MultinomialNB()

In [17]:
nb.score(X_test_cv,y_test)

0.85504

* ngram_range = (1,2)

In [18]:
cvect2 = CountVectorizer(ngram_range=(1,2),stop_words='english')
cvect2.fit(X_train)
%time X_train_cv2 = cvect2.transform(X_train)
print(X_train_cv2.shape)

CPU times: user 5.21 s, sys: 21.2 ms, total: 5.23 s
Wall time: 5.24 s
(18750, 1386558)


In [19]:
X_test_cv2 = cvect2.transform(X_test)
X_test_cv2.shape

(6250, 1386558)

In [20]:
nb2 = MultinomialNB()
nb2.fit(X_train_cv2,y_train)

MultinomialNB()

In [21]:
nb2.score(X_test_cv2,y_test)

0.8672

* 모델 저장하고 불러오기

In [22]:
import joblib
joblib.dump(cvect2,'imdb_cvect1.pkl')
joblib.dump(nb2,'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [32]:
#데이터 로드
new_cvect = joblib.load('imdb_cvect1.pkl')
new_nb = joblib.load('imdb_nb2.pkl')

* 실제 데이터로 검증

In [33]:
review = '''
Robert P is a terrible Batman. Absolutely no Bat-charisma at all. 
The Riddler character is a complete bore! I hated all of this film. 
I certainly do not understand the good reviews. 
It's not close to the Dark Knight caliber. Please do not make a sequel.
'''

In [34]:
#텍스트 전처리
import re
review =re.sub('[^[A-Za-z]',' ',review).strip()
review_cv = new_cvect.transform([review])
review_cv.shape

(1, 1386558)

In [35]:
new_nb.predict(review_cv)

array([0])